In [1]:
!pip install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 626.8 kB/s eta 0:00:00


In [11]:
# 训练简单的SVM模型来分类心跳数据
def train_svm():
    # 正常心跳数据 (90-120)
    X_normal = np.random.randint(90, 121, (50, 1))
    y_normal = np.zeros(50)  # 正常

    # 异常心跳数据
    X_abnormal = np.concatenate([np.random.randint(60, 90, (25, 1)), np.random.randint(121, 150, (25, 1))])
    y_abnormal = np.ones(50)  # 异常

    # 合并数据
    X = np.vstack((X_normal, X_abnormal))
    y = np.concatenate((y_normal, y_abnormal))

    model = svm.SVC(kernel='linear')
    model.fit(X, y)
    return model

model = train_svm()


In [13]:
import paho.mqtt.client as mqtt

broker_address = "broker.emqx.io"
port = 1883
topic = "patient/heartbeat"
# username = "test"
# password = "1"

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to broker")
        client.subscribe(topic)
    else:
        print("Connection failed with code", rc)

def on_message(client, userdata, message):
    heartbeat = int(message.payload.decode("utf-8"))
    print(f"Received heartbeat: {heartbeat}")
        # 预测心跳状态
    prediction = model.predict([[heartbeat]])
    if prediction == 0:
        print("Heartbeat is normal")
    else:
        print("Heartbeat is abnormal")

client = mqtt.Client()
# client.username_pw_set(username, password)
client.on_connect = on_connect
client.on_message = on_message

client.connect(broker_address, port)
client.loop_forever()


<ipython-input-13-6476dca205a6>:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected to broker
Received heartbeat: 112
Heartbeat is normal
Received heartbeat: 114
Heartbeat is normal
Received heartbeat: 84
Heartbeat is abnormal
Received heartbeat: 116
Heartbeat is normal
Received heartbeat: 85
Heartbeat is abnormal
Received heartbeat: 122
Heartbeat is normal
Received heartbeat: 126
Heartbeat is normal
Received heartbeat: 120
Heartbeat is normal
Received heartbeat: 75
Heartbeat is abnormal
Received heartbeat: 120
Heartbeat is normal
Received heartbeat: 75
Heartbeat is abnormal
Received heartbeat: 66
Heartbeat is abnormal
Received heartbeat: 116
Heartbeat is normal
Received heartbeat: 104
Heartbeat is normal
Received heartbeat: 70
Heartbeat is abnormal


KeyboardInterrupt: 